# Evalution and Analysis

## Imports:

In [1]:
import pandas as pd
from pathlib import Path
import re
import numpy as np
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

## Get predictions and metrics:

In [2]:
def get_model_name(fileName:str):
    '''
    This function gets the model name according to the following naming convention: {model name}_{information...}.npy,
    where {information} is optional. 

    Parameters:
    -----------
    fileName: A string of the models predictions file name.

    Returns:
    --------
    A string of the models name.
    '''

    return re.findall(r'([a-zA-Z0-9]+)',fileName)[0]


def get_predictions(predsPath:Path):
    '''
    This function combines all predictions of all models into one dataframe and includes the ground truth.

    Parameters:
    -----------
    predsPath: A path/string of a path, to the directory of all the predictions

    Returns:
    --------
    Returns a pandas Dataframe of all the models predictions, as well as the ground truth. 
    '''
    # Get the ground truth:
    groundTruth= pd.read_csv(Path('../Data/Test/test1990s.csv'),parse_dates=[0],date_format='%m%Y',index_col=0, usecols=[0,1])

    # make an empty Dataframe to store the models predictions (making the index  the observation data):
    predsDf= pd.DataFrame(index=groundTruth.index)

    # Add the ground truth to the predictions dataframe
    predsDf['ground_truth']= groundTruth

    # Loop over all the files in the predictions folder:
    for i in list(predsPath.glob('*.npy')):
        # Add the predictions to the predictions dataframe, where the column is the model name
        predsDf[get_model_name(i.name)]= np.load(i)[:48]
    
    return predsDf

def calc_metrics(predictionsDf:pd.DataFrame, horizon = None):
    '''
    This function calculates the evaluation metrics of each model, given the predictions dataframe.
    The following metrics are used:
        * RMSE
        * MAE
        * r^2 score

    Parameters:
    -----------
    predictionsDf: a pandas datframe containg the ground truth and all the predictions for each model, organized in columns.

    horizon: number of timesteps to calculate the metrics.

    Returns:
    --------
    Returns a pandas Dataframe containg all the evaluation metrics of all the models, where each column represents a metric and each row represents a model.
    '''
    # Deafult to horizon of 2 years:
    if horizon is None:
        horizon= predictionsDf.shape[0]

    # create an empty dataframe with columns reprresnting an evaluation metric
    metricsDf= pd.DataFrame(columns=['RMSE','MAE', 'r2'])
    
    # Loop over all columns/models in the prtedictions dataframe
    for model in predictionsDf.columns.drop('ground_truth'):#
        # Calculate the metrics and add them to the metrics dataframe
        metricsDf.loc[model,'RMSE']=root_mean_squared_error(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])
        metricsDf.loc[model,'MAE']=mean_absolute_error(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])
        metricsDf.loc[model,'r2']=r2_score(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])

    return metricsDf

In [3]:
predsPath= Path('../Predictions/')
predsDf=get_predictions(predsPath)
display(predsDf)

,ground_truth,Naive,SARIMAX
observation_date,,,
01/2021,106.083,105.69,106.038947
02/2021,106.471,105.69,106.501714
03/2021,107.032,105.69,106.877229
04/2021,107.632,105.69,107.086592
05/2021,108.184,105.69,107.423990
06/2021,108.748,105.69,107.791156
07/2021,109.275,105.69,108.105803
08/2021,109.720,105.69,108.381933
09/2021,110.047,105.69,108.524800


# Calculate Metrics:

## Horizon = 1

In [9]:
# WARNING: r2 is not well defined for less than 2 samples
metrics_1= calc_metrics(predsDf, horizon=1).drop('r2', axis=1)
display(metrics_1)

c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,RMSE,MAE
Naive,0.393,0.393
SARIMAX,0.044053,0.044053


## Horizon = 3

In [5]:
metrics_3= calc_metrics(predsDf, horizon=3)
display(metrics_3)

,RMSE,MAE,r2
Naive,0.92473,0.838667,-4.63461
SARIMAX,0.094583,0.076513,0.941053


## Horizon = 6

In [6]:
metrics_6= calc_metrics(predsDf, horizon=6)
display(metrics_6)

,RMSE,MAE,r2
Naive,1.910852,1.668333,-3.206534
SARIMAX,0.550375,0.4153,0.651031


## Horizon = 12

In [7]:
metrics_12= calc_metrics(predsDf, horizon=12)
display(metrics_12)

,RMSE,MAE,r2
Naive,3.786199,3.282,-3.022498
SARIMAX,1.446428,1.142871,0.41294


# Analysis: